In [1]:
import pymolviz as pmv
import numpy as np

In [2]:
meshes = []
meshes.append(pmv.Sphere(np.array([0.5, 0.3, 0.5]), 1))
meshes.append(pmv.Sphere(np.array([0.7, 0.4, 1]), 1))
meshes.append(pmv.Cylinder(np.array([-1, -1, -2]), np.array([1, 1.5, 1]), 0.5, "red"))
bsp_trees = [pmv.BSP_Node.from_mesh(mesh) for mesh in meshes]

In [3]:
bsp_trees[0].to_mesh()

In [4]:
union = bsp_trees[0].union(bsp_trees[1])
subtraction = bsp_trees[1].subtract(bsp_trees[2])
intersection = bsp_trees[0].intersect(bsp_trees[1])

In [5]:
print([len(node.all_polygons()) for node in bsp_trees])

[1051, 1090, 120]


cur_level = [node1]
next_level = []
plane_meshes = []
level_index = 0
fronts = []
backs = []
while len(cur_level) > 0:
    cur_meshes = []
    front_meshes = []
    back_meshes = []
    for node in cur_level:
        if node.front is not None:
            if level_index < 4:
                front_meshes.extend([p.to_mesh() for p in node.front.all_polygons()])
            next_level.append(node.front)
        if node.back is not None:
            if level_index < 4:
                back_meshes.extend([p.to_mesh() for p in node.back.all_polygons()])
            next_level.append(node.back)
        cur_meshes.append(pmv.Sphere(node.position, 0.1, "red"))
        cur_meshes.append(node.plane_mesh())
    fronts.append(pmv.Collection(front_meshes, name = f"front_{level_index}"))
    backs.append(pmv.Collection(back_meshes, name = f"back_{level_index}"))
    plane_meshes.append(pmv.Collection(cur_meshes, name = f"level_{level_index}"))
    cur_level = next_level
    level_index += 1
    next_level = []

In [6]:
reverted_meshes = [bsp_tree.to_mesh() for bsp_tree in bsp_trees]
normal_meshes = [bsp_tree.to_normal_arrows() for bsp_tree in bsp_trees]

In [7]:
union_collection = pmv.Collection([union.to_mesh()], "union")
union_wireframe = pmv.Collection([union.to_mesh().to_wireframe()], "union_wireframe")
intersection_collection = pmv.Collection([intersection.to_mesh()], "intersection")
intersection_wireframe = pmv.Collection([intersection.to_mesh().to_wireframe()], "intersection_wireframe")
difference_collection = pmv.Collection([subtraction.to_mesh()], "difference")
difference_wireframe = pmv.Collection([subtraction.to_mesh().to_wireframe()], "difference_wireframe")

In [8]:
reverted_collections = [pmv.Collection([reverted_mesh], f"reverted_mesh_{i}") for i, reverted_mesh in enumerate(reverted_meshes)]
original_collections = [pmv.Collection([mesh], f"original_mesh_{i}") for i, mesh in enumerate(meshes)]
normal_collections = [pmv.Collection([mesh], f"polygon_normals_{i}") for i, mesh in enumerate(normal_meshes)]
wireframe_collections = [pmv.Collection([reverted_mesh.to_wireframe()], f"reverted_wireframe_{i}") for i, reverted_mesh in enumerate(reverted_meshes)]
script = pmv.Script([union_collection, union_wireframe, intersection_collection, intersection_wireframe,  difference_collection, difference_wireframe, *normal_collections, *reverted_collections, *original_collections, *wireframe_collections])
script.write("testMerge.py")